In [3]:
import pandas as pd
import torch

In [ ]:
data=pd.read_csv("dataset.csv")

data.head()

In [ ]:
import matplotlib.pyplot as plt
ax=data.hist(bins=100, figsize=(36,18))
plt.savefig("histogram.pdf")

In [ ]:
import numpy as np
np.floor(155/13)

In [ ]:
a._data['Filtered_Accelerator_Pedal_value']

In [ ]:
np.sum(a._data.isna())

In [4]:
import numpy as np
from sklearn import preprocessing
from torch.utils.data import Dataset

class DatasetDrivers(Dataset):
    def __init__(self, file_path='dataset.csv', classes_to_drop=['Class', 
                                                                 'PathOrder', 
                                                                 'Time(s)', 
                                                                 'Filtered_Accelerator_Pedal_value',
                                                                 'Inhibition_of_engine_fuel_cut_off',
                                                                 'Fuel_Pressure',
                                                                 'Torque_scaling_factor(standardization)',
                                                                 'Glow_plug_control_request'], window_size=16, normalize=True, normalize_method='mean_std'):
        print("Loading")
        self._window_size=window_size
        self._data=pd.read_csv(file_path)
        self._data.sort_values(by="Class", inplace=True)
        print("Removing extra columns, initial data size is:")
        print(len(self._data))
        n_per_class=[]
        for class_uniq in list(self._data['Class'].unique()):
            tot_number=sum(self._data['Class']==class_uniq)
            n_per_class.append(tot_number-tot_number%window_size)
            to_drop=tot_number%window_size
            index_to_start_removing=self._data[self._data['Class']==class_uniq].index[0]
            #import pdb; pdb.set_trace()
            self._data.drop(self._data.index[index_to_start_removing:index_to_start_removing+to_drop],inplace=True)
        print("After removing, the size becomes:")
        print(len(self._data))
        self._data.reset_index(inplace=True)
        #import pdb; pdb.set_trace()

        index_starting_class=[]
        #index_starting_class.append(0)
        for class_uniq in list(self._data['Class'].unique()):
            index_starting_class.append(self._data[self._data['Class']==class_uniq].index[0])
        #import pdb; pdb.set_trace()
        index_starting_class.append(len(self._data))
        sequences=[]
        for i in range(len(index_starting_class)):
            if i!=len(index_starting_class)-1:#index_starting_class[-1]:
                ranges=np.arange(index_starting_class[i], index_starting_class[i+1])
            
            for j in range(0,len(ranges),int(self._window_size/2)):
                if len(ranges[j:j+self._window_size])==16:
                    sequences.append(ranges[j:j+self._window_size])
        self._sequences=sequences
        
        print("Drop columns: ")
        print(classes_to_drop)
        self._labels=self._data['Class'].values
        self._data.drop(classes_to_drop, inplace=True, axis=1)
        if normalize:
            for col in self._data.columns:
                if normalize_method=='min_max':
                    self._data[col]=(self._data[col]-self._data[col].min())/(self._data[col].max()-self._data[col].min())
                elif normalize_method=="mean_std":
                    self._data[col]=(self._data[col]-self._data[col].mean())/(self._data[col].std())
        X=np.empty((len(sequences), self._window_size, len(self._data.columns)))
        y=[]#np.empty((len(sequences), ))
        for n_row, sequence in enumerate(sequences):
            X[n_row,:,:]=self._data.iloc[sequence]
            y.append(self._labels[sequence[0]])
        assert len(y)==len(X)
        self._X=X
        targets = preprocessing.LabelEncoder().fit_transform(y)
        targets = torch.as_tensor(targets)
        #targets_tensor = torch.nn.functional.one_hot(targets.to(torch.int64), num_classes=10)
        self._y=targets
    
    def __len__(self):
        return len(self._X)
    
    def __getitem__(self, index):
        return torch.FloatTensor(self._X[index,:,:]), self._y[index]


In [5]:
a=DatasetDrivers()

Loading
Removing extra columns, initial data size is:
94380
After removing, the size becomes:
94320
Drop columns: 
['Class', 'PathOrder', 'Time(s)', 'Filtered_Accelerator_Pedal_value', 'Inhibition_of_engine_fuel_cut_off', 'Fuel_Pressure', 'Torque_scaling_factor(standardization)', 'Glow_plug_control_request']


In [7]:
train_size = int(0.85 * len(a))
val_size = int(0.05 * len(a))
test_size = len(a)-train_size-val_size#int(0.10 * len(a))

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(a, [train_size,val_size, test_size])

batch_size = 32
n_workers=0

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True,
                                           drop_last=True,
                                           num_workers=n_workers)
 
validation_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False,
                                           drop_last=True,
                                           num_workers=n_workers)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False,
                                          num_workers=n_workers)

# The shape of the input element is batch,sequence,features
# The input to the network must have batch_first = True

class NEURAL(torch.nn.Module):
    
    def __init__(self, batch_size, window_size, num_features):
        super(NEURAL, self).__init__()
        self.lstm1 = torch.nn.LSTM(num_features, 160,batch_first = True)
        self.lstm2 = torch.nn.LSTM(160, 200, 1)
        
        # I need to get the output of last hidden layer 
        # What size is it DONT KNOW
        self.fc = torch.nn.Linear(16, 10)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):

        lstm1_out, (h_t1, c_t1) = self.lstm1(x) 
        #print("shape of lstm1 is: ",lstm1_out.shape)
        #print("shape of lstm1 h is: ",h_t1.shape)        
        #print("shape of lstm1 c is: ",c_t1.shape)   
        
        lstm2_out, (h_t2, c_t2) = self.lstm2(lstm1_out)      
        #print("shape of lstm2 is: ",lstm2_out.shape)
        #print("shape of lstm2 h is: ",h_t2.shape)        
        #print("shape of lstm2 c is: ",c_t2.shape)
        #print("shape of lstm2 c is: ",h_t2[:,:,-1].shape)
              
        fc_out = self.fc(lstm2_out[:,:,-1])
        out = self.sigmoid(fc_out)

        return out
    
    
inputs, classes = next(iter(train_loader)) 

model = NEURAL(inputs.shape[0],inputs.shape[1],inputs.shape[2])

learning_rate = 1e-4
num_epochs = 500

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
lambda1 = lambda epoch: 0.65 ** epoch
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)


def train_one_epoch():
    running_loss = 0

    for i, (data, labels) in enumerate(train_loader):
        
        #print(data)
        #print("labels type",type(labels))
        # Zero your gradients for every batch!
        optimizer.zero_grad()
        # Make predictions for this batch
        outputs = model(data)
        #print("the output has shape: ", outputs)
        # Compute the loss and its gradients
        loss = criterion(outputs,labels)
        loss.backward()
        
        # Compute the loss and its gradients
        optimizer.step()
 
        running_loss += loss.item()
        #print(i)
        if i % 100 == 99:
            last_loss = running_loss / 100 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.

    return last_loss

epoch_number = 0

for epoch in range(num_epochs):

    #print('EPOCH {}:'.format(epoch_number + 1))

    model.train(True)
    avg_loss = train_one_epoch()
    torch.cuda.empty_cache()

    model.train(False)

    running_vloss = 0.0

    for i, vdata in enumerate(validation_loader):
        vinputs, vlabels = vdata
        voutputs = model(vinputs)
        vloss = criterion(voutputs, vlabels[:,-1,:])
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)

    
    torch.cuda.empty_cache()

    with open('res.csv', 'a') as f:
        f.write('{},{},{} \n'.format(epoch_number + 1,avg_loss, avg_vloss))

    print('{:d},{},{}'.format(epoch_number + 1,avg_loss, avg_vloss))
    epoch_number += 1

  batch 100 loss: 2.3002632737159727
  batch 200 loss: 2.294803659915924
  batch 300 loss: 2.284939749240875


IndexError: too many indices for tensor of dimension 1

In [ ]:
inputs, classes = next(iter(train_loader)) 
inputs.shape

In [ ]:
classes

In [ ]:

# The shape of the input element is batch,sequence,features
# The input to the network must have batch_first = True

for it in train_loader:
    print(np.shape(it[0]))

In [ ]:
a._X[:10,:,:]

In [ ]:
a._sequences

In [ ]:
indexes=range(10)
window=4
s=[]
for i in np.arange(0,len(indexes),int(window/2)):
    s.append(indexes[i:i+window])

In [ ]:
s

In [ ]:
tot_per_class=..
n_per_class=tot_per_class/window/2
X_array=np.empy((n_per_class, features, window))
y_array=np.empty((n_per_class, ))

indexes_starting_class=..
for i in indexes_starting_class:
    if i!=indexes_starting_class[-1]:
        ranges=np.arange(index_starting_class[i], index_starting_class[i+1])
    else:
        ranges=np.arange(index_starting_class[i], len(data))
    
data.iloc[:10].to_numpy()

In [ ]:
window_size=13
sequences=...
data_new=pd.DataFrame()
